<a href="https://colab.research.google.com/github/minit88/ML-DL/blob/main/Lab_10/2_mnist_deep_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.init

In [18]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
  torch.cuda.manual_seed_all(777)

In [19]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size=100

In [20]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                          train=False, # False 로 해야지 test가 들어옴
                          transform=transforms.ToTensor(),
                          download=True)

In [21]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size = batch_size,
                                          shuffle=True,
                                          drop_last = True)


In [22]:
class CNN(nn.Module):

  def __init__(self):
    super(CNN,self).__init__()
    self.layer1 = nn.Sequential(nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1),
                                nn.ReLU(),
                                nn.MaxPool2d(2)
    )
    self.layer2 = nn.Sequential(nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
                                nn.ReLU(),
                                nn.MaxPool2d(2)
    )
    self.fc=nn.Linear(7*7*64,10,bias = True)
    torch.nn.init.xavier_uniform_(self.fc.weight)# 레이어 초기화

  def forward(self,x):
    out = self.layer1(x)
    out = self.layer2(out)

    out = out.view(out.size(0),-1)
    out = self.fc(out)
    return out

In [23]:
model=CNN().to(device)

In [24]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr= learning_rate)

In [26]:
# training
total_batch = len(data_loader) # 전체 배치사이즈가 몇인지를 알 수 있다
print('Learning start')

for epoch in range(training_epochs):
  avg_cost = 0

  for X,Y in data_loader: # X = image , Y = label
    X = X.to(device) # cuda 연산진행
    Y = Y.to(device) # cuda 연산진행

    hypothesis = model(X)
    cost = criterion(hypothesis,Y)


    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    avg_cost +=cost/total_batch

  print('[Epoch:{}] cost = {}'.format(epoch+1,avg_cost))  
print("Learing Finished")

Learning start
[Epoch:1] cost = 0.06304250657558441
[Epoch:2] cost = 0.04638969898223877
[Epoch:3] cost = 0.03748450055718422
[Epoch:4] cost = 0.03143305703997612
[Epoch:5] cost = 0.026136364787817
[Epoch:6] cost = 0.022024285048246384
[Epoch:7] cost = 0.018365776166319847
[Epoch:8] cost = 0.016204392537474632
[Epoch:9] cost = 0.013407928869128227
[Epoch:10] cost = 0.010212686844170094
[Epoch:11] cost = 0.009767561219632626
[Epoch:12] cost = 0.008789300918579102
[Epoch:13] cost = 0.0062956116162240505
[Epoch:14] cost = 0.005553173832595348
[Epoch:15] cost = 0.006873254664242268
Learing Finished


In [30]:
with torch.no_grad():
  X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)
  
  prediction = model(X_test)
  Correct_prediction = torch.argmax(prediction,1)==Y_test
  accuracy = Correct_prediction.float().mean()
  print('Accuracy:',accuracy.item())



Accuracy: 0.9848999977111816
